In [14]:
Experiments_file='/home/kate/Research/YearBuilt/Experiments/Experiments.xlsx'
AllExperiments_tab='Experiments'
Experiment_name='Prediction'
#Experiment configuration: differenet datasets to try
#each line in the file contains the model name and set of features to built a dataset for SageMaker
Experiment_tab='%s Models'%Experiment_name

#Looks like Trial name should be unique in my environment, not in the experiment it belongs to
Trial_name='%s-PreparingData'%Experiment_name



#original dataset was created from a Redshift query and uploaded into S3
bucket='kdproperty'
path_to_data_file='/Data/'
data_file='property_basedata_v3.csv'

split_year='2020'


instance_type='ml.t3.medium'
instance_count=1

In [2]:
import boto3
import sys
import time
import pandas as pd
import numpy as np

import sagemaker
from sagemaker import get_execution_role
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

from sagemaker.analytics import ExperimentAnalytics

In [3]:
sys.path.append('/home/kate/Research/YearBuilt/Notebooks/Property')
import ExperimentsUtils as eu

In [4]:
#delete experiment and trials but not output files associated with jobs in experiments
eu.cleanup_experiment(Experiment_name)

In [5]:
eu.create_experiment(Experiment_name)
eu.create_trial(Experiment_name,Trial_name)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


In [7]:
region = boto3.session.Session().region_name
role = 'arn:aws:iam::757107622481:role/service-role/AmazonSageMaker-ExecutionRole-20200819T131882'
sagemaker_session = sagemaker.session.Session(default_bucket=bucket)

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


'kdproperty'

In [8]:
%%writefile preprocessingTestingData.py

#Training and Validation dataset for SageMaker are the same structure: no headers, the first column is a target and the rest are features


import argparse
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

if __name__=='__main__':
    
    parser = argparse.ArgumentParser()
    parser.add_argument('--data_file', type=str)
    parser.add_argument('--split_year', type=int)     
    parser.add_argument('--featureset', type=str)    
    args, _ = parser.parse_known_args()    
    print('Received arguments {}'.format(args))
    
    featureset=args.featureset.split(',')
    input_data_path = os.path.join('/opt/ml/processing/input', args.data_file)
    output_data_path = os.path.join('/opt/ml/processing/output/data', 'data.csv')    

  
    
   
    
    print('Reading input data from {}'.format(input_data_path))
    dataset = pd.read_csv(input_data_path, error_bad_lines=False, index_col=False)
    
    
    dataset=dataset[(dataset.cal_year == args.split_year)]
    
    X = pd.DataFrame()
    for f in featureset:
        X[f]=dataset.eval(f)
    

    X.to_csv(output_data_path, header=False, index=False)
    
 
    

Overwriting preprocessing.py


In [10]:
processors=list()
#take into account only one,first row 
for index, row in models.iterrows():
    model=row['Model']
    print (index, ': Creating datasets for model %s'%model)
    featureset=row[1:51].tolist()
    featureset=[x for x in featureset if str(x) != 'nan']
    print(','.join(featureset))
    data_processor = SKLearnProcessor(framework_version='0.20.0',
                                     role=role,
                                     instance_type=instance_type,
                                     instance_count=instance_count,
                                     tags=[{'Key': 'Model', 'Value':model},
                                           {'Key': 'Featureset', 'Value': ':'.join(featureset)},
                                           {'Key': 'split_year','Value':split_year},
                                           {'Key': 'val_size','Value':val_size},])

    data_processor.run(code='preprocessingTestingData.py',
                        inputs=[ProcessingInput(
                        source='s3://%s%s'%(bucket,path_to_data_file+data_file),
                        destination='/opt/ml/processing/input')],
                        outputs=[ProcessingOutput(output_name='data', source='/opt/ml/processing/output')                                                       
                                ],
                        arguments=['--data_file',data_file,
                                 '--split_year',split_year,                             
                                 '--featureset', ','.join(featureset).replace(' ','')],
                       experiment_config = {
        'ExperimentName': Experiment_name ,
        'TrialName' : Trial_name,
        'TrialComponentDisplayName' : '%s-%s'%(Trial_name,model.replace('_','-')),},
                    wait=False
                     )
    processors.append(data_processor)
    break

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


0 : Creating datasets for model StartModel
usagetype_encd,sqft


INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2020-12-17-16-24-46-358



Job Name:  sagemaker-scikit-learn-2020-12-17-16-24-46-358
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://kdproperty/Data/property_basedata_v3.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-17-16-24-46-358/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'training_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-17-16-24-46-358/output/training_data', 'LocalPath': '/opt/ml/processing/output/training_data', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


...


Job Name:  sagemaker-scikit-learn-2020-12-17-16-24-47-575
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://kdproperty/Data/property_basedata_v3.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-17-16-24-47-575/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'training_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-17-16-24-47-575/output/training_data', 'LocalPath': '/opt/ml/processing/output/training_data', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


2 : Creating datasets for model LimitDeductible
usagetype_encd,sqft,yearbuilt  ,cova_deductible,cova_limit 


INFO:sagemaker:Creating processing-job with name sagemaker-scikit-learn-2020-12-17-16-24-49-691



Job Name:  sagemaker-scikit-learn-2020-12-17-16-24-49-691
Inputs:  [{'InputName': 'input-1', 'S3Input': {'S3Uri': 's3://kdproperty/Data/property_basedata_v3.csv', 'LocalPath': '/opt/ml/processing/input', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}, {'InputName': 'code', 'S3Input': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-17-16-24-49-691/input/code/preprocessing.py', 'LocalPath': '/opt/ml/processing/input/code', 'S3DataType': 'S3Prefix', 'S3InputMode': 'File', 'S3DataDistributionType': 'FullyReplicated', 'S3CompressionType': 'None'}}]
Outputs:  [{'OutputName': 'training_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-2-757107622481/sagemaker-scikit-learn-2020-12-17-16-24-49-691/output/training_data', 'LocalPath': '/opt/ml/processing/output/training_data', 'S3UploadMode': 'EndOfJob'}}, {'OutputName': 'validation_data', 'S3Output': {'S3Uri': 's3://sagemaker-us-west-

In [11]:
#wait till the rest of processing jobs complete
eu.wait_processing_jobs(processors=processors,check_every_sec=10,print_every_n_output=10,wait_min=60)

Processing job sagemaker-scikit-learn-2020-12-17-16-24-46-358 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-47-575 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-49-691 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-17-16-24-46-358 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-47-575 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-49-691 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-17-16-24-46-358 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-47-575 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-49-691 status: InProgress
Continue waiting...
Processing job sagemaker-scikit-learn-2020-12-17-16-24-46-358 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-16-24-47-575 status: InProgress
Processing job sagemaker-scikit-learn-2020-12-17-

When all jobs are done I want to have the list of training and validation data created in each job/trail component

In [12]:
#no way to see analitics for a specific trial using SageMaker Python SDK
#It's possible to have more then one trial to prepare data in each experiments
#Or they plan one experiment - one trial only?
#Can filter by DisplayName if it includes Trial Name

trial_component_analytics = ExperimentAnalytics(
    experiment_name=Experiment_name
)
trial_comp_ds = trial_component_analytics.dataframe()
trial_ds=trial_comp_ds[trial_comp_ds['DisplayName'].str.contains(Trial_name)].copy()
trial_ds['Model']=trial_ds['DisplayName'].str.replace(Trial_name+'-','')
trial_ds['Model']=trial_ds['Model'].str.replace('-','_')
trial_ds=trial_ds[['Model','DisplayName','data - Value'']]
trial_ds.columns=['Model','Trial Component','Data']
trial_ds

INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials


,Model,Trial Component,Training_data,Validation_data
0,LimitDeductible,BaseFeatures-PreparingTrainValidData-LimitDedu...,s3://sagemaker-us-west-2-757107622481/sagemake...,s3://sagemaker-us-west-2-757107622481/sagemake...
1,YearBuilt,BaseFeatures-PreparingTrainValidData-YearBuilt,s3://sagemaker-us-west-2-757107622481/sagemake...,s3://sagemaker-us-west-2-757107622481/sagemake...
2,StartModel,BaseFeatures-PreparingTrainValidData-StartModel,s3://sagemaker-us-west-2-757107622481/sagemake...,s3://sagemaker-us-west-2-757107622481/sagemake...


In [13]:
#Saving into the Experiment log file names of created dataset in S3 to train models in other module 
eu.SaveToExperimentLog(Experiments_file, '%s InputData'%Experiment_name, trial_ds)